# My first Jupyter Notebook : CMS PhaseII Tk Trigger Work 

If you would like to see the code used to generate the notebook below then click on show code! By default all code snippets are hidden to make this notebook more legible.

In [9]:
from IPython.display import HTML

HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

### Study 
Using [TMTT Software](https://svnweb.cern.ch/cern/wsvn/UK-TrackTrig/software/cmssw/trunkSimpleCode9/README_SimpleCode.txt) to understand the effect of the choice of digitization (for stubs and track parameters) on an algorithm's resolution. 

---

### Method 
This was done by reducing the number of bits in the configuration parameters of two **TMTrackProducer** processes : 

* **TrackDigi** 
    - RtBits , ZBits, BendBits, PhiSectorBits, PhiSBits, PhiOBits
* **StubDigitize** 
    - d0Bits , oneOver2rBits , phi0Bits , z0Bits , tanlambdaBits , chisquaredBits
    
and looking at the changes in the resolution of ($z0$,$d0$,$\eta$,$\phi$) 
for the two fitting algorithms KF4ParamsComb,KF5ParamsComb.

### Monte Carlo 
Monte Carlo samples used in this study were made with the D17 (=T5) tilted tracker geometry and use Seb Viret's Dec. 2017 stub window sizes which you can find in CMSSW 9.3.7 in file L1Trigger/TrackTrigger/python/TTStubAlgorithmRegister_cfi.py and consist of 1000 $t\bar{t}$ events with PU200.

---

### Analyis Code 

Now some code to look at the results of the study.

---

In [3]:
import ctypes
import ROOT
import QuickLook as QL
%jsroot on

Welcome to JupyROOT 6.12/06


In [2]:
%%cpp
class DTCdigis
{
    private :
        std::unordered_map<std::string,uint32_t> _map_ = {{"Rt",12},{"Z",14},{"Bend",6},{"PhiSector",6},{"PhiS",13},{"PhiO",15}};
    public : 
        DTCdigis(){;}
        uint32_t returnNbits(std::string cKey) {auto it = _map_.find(cKey); return (it != _map_.end()) ?  it->second : 0 ;}
        void changeNbits(std::string cKey, uint32_t cNbits){ auto it = _map_.find(cKey); if (it != _map_.end()) it->second = cNbits ; }
        std::string getKey(size_t cIndex)
        {
            std::vector<std::string> retval; 
            for (auto const& element : _map_) 
            {
                retval.push_back(element.first);
            }
            return retval[cIndex];
        }
        size_t getSize(){return _map_.size();}
        
};
class KFdigis
{
    private :
        std::unordered_map<std::string,uint32_t> _map_ = {{"d0",12},{"oneOver2r",18},{"phi0",18},{"z0",18},{"tanlambda",18},{"chisquared",17}};
    public : 
        KFdigis(){;}
        uint32_t returnNbits(std::string cKey) {auto it = _map_.find(cKey); return (it != _map_.end()) ?  it->second : 0 ;}
        void changeNbits(std::string cKey, uint32_t cNbits){ auto it = _map_.find(cKey); if (it != _map_.end()) it->second = cNbits ; }
        std::string getKey(size_t cIndex)
        {
            std::vector<std::string> retval; 
            for (auto const& element : _map_) 
            {
                retval.push_back(element.first);
            }
            return retval[cIndex];
        }
        size_t getSize(){return _map_.size();}
        
};

In [35]:
def drawScanResult_DTC(pDigiVarName, pDigiScanRange,pVarName ) :
    cCanvasName = 'cScan%s_%s' % (pDigiVarName,pVarName)
    cCanvas = ROOT.gROOT.FindObject(cCanvasName)
    if(not cCanvas):
        cCanvas = ROOT.TCanvas(cCanvasName,"",350*2,350)
        cCanvas.Divide(3,1)
    else : 
        cCanvas.Clear()
        cCanvas.Divide(3,1)
        
    cDTCdigi = ROOT.DTCdigis()
    cDefValue = cDTCdigi.returnNbits(pDigiVarName)
    cSteps = pDigiScanRange[1] - pDigiScanRange[0]
    cProfName='hResolution%s_%s'%(pDigiVarName,pVarName)
    cProfTitle='Resolution [%s scan]; Number of %s bits; #sigma_{%s}'%(pDigiVarName,pDigiVarName,pVarName)
    cProfResolution = ROOT.gROOT.FindObject(cProfName)
    if(cProfResolution) :
        cProfResolution.Delete()
    cProfResolution = ROOT.TH1D(cProfName,cProfTitle,cSteps+1,cDefValue+pDigiScanRange[0],cDefValue+pDigiScanRange[1]+1)
    cProfResolution.GetYaxis().SetTitleOffset(1.2)
    
    cProfName='hEfficiency%s'%(pDigiVarName)
    cProfTitle='Efficiency [%s scan]; Number of %s bits; #epsilon'%(pDigiVarName,pDigiVarName)
    cProfEfficiency = ROOT.gROOT.FindObject(cProfName)
    if(cProfEfficiency) :
        cProfEfficiency.Delete()
    cProfEfficiency = ROOT.TH1D(cProfName,cProfTitle,cSteps+1,cDefValue+pDigiScanRange[0],cDefValue+pDigiScanRange[1]+1)
    cProfEfficiency.GetYaxis().SetTitleOffset(1.2)


    cHists=list()
    cHistName='hResolution%s_%s_2D' %(pDigiVarName, pVarName)
    cResolution2D = ROOT.gROOT.FindObject(cHistName)
    if(cResolution2D) :
        cResolution2D.Delete()
    
    cDigiNames=['Rt','Z','Bend','PhiSector','PhiS','PhiO']
    for cDigiValue in range(cDefValue+pDigiScanRange[0],cDefValue+pDigiScanRange[1]+1) : 
        #cFileName='./data/Hists_%dRtBits_%dZBits_%dBendBits_%dPhiSectorBits_%dPhiSBits_%dPhiOBits.root' % (cDigiValue, QL.TMTT_defZ_bits, QL.TMTT_defBend_bits, QL.TMTT_defPhiSector_bits, QL.TMTT_defPhiS_bits, QL.TMTT_defPhiO_bits)
        cFileName='./data/Hists'
        cHistName='hResolutionVs%s_'%(pVarName)
        for (cIndex,cDigiName) in enumerate(cDigiNames) : 
            cKeyValue = cDTCdigi.returnNbits(cDigiName)
            if( cDigiName == pDigiVarName ) : 
                cKeyValue=cDigiValue
            cFileName='%s_%d%sBits' % (cFileName,cKeyValue,cDigiName)
            cHistName='%s_%d%sBits' % (cHistName,cKeyValue,cDigiName)
        cFileName='%s.root'%(cFileName)
        cEff = QL.getAbsoluteAlgEfficiency(cFileName, "eta", "KF4ParamsComb")
        cHists.append(QL.getFoldedResolution(cFileName, pVarName, "KF5ParamsComb").Clone(cHistName))
        cHistIndex=len(cHists)-1
        cResolution = cHists[cHistIndex].GetRMS()
        cResolutionError = cResolution/(2*(cHists[cHistIndex].GetEntries()-1))**0.5
        #print('%s = %d bits : Resolution = %.3f ± %.3f, Eff = %.3f ± %.3f '%(pDigiVarName, cDigiValue, cResolution,cResolutionError,cEff[0]*100,cEff[1]*100))
        cHists[cHistIndex].SetStats(0)
        cHists[cHistIndex].GetXaxis().SetTitle("#eta_{0,KF} - #eta_{0,TP}")
        cHists[cHistIndex].GetXaxis().SetTitleOffset(1.2)
        cHists[cHistIndex].GetYaxis().SetRangeUser(0,25e3)
        cHists[cHistIndex].SetLineColor(cHistIndex+1)
        cHists[cHistIndex].SetFillStyle(3001)
        cHists[cHistIndex].SetFillColor(cHistIndex+1)

        cHistName='hResolution%s_%s_2D' %(pDigiVarName, pVarName)
        cResolution2D = ROOT.gROOT.FindObject(cHistName)
        cNbinsX = cProfResolution.GetXaxis().GetNbins()
        cXmin=cProfResolution.GetXaxis().GetXmin()
        cXmax=cProfResolution.GetXaxis().GetXmax()
        cNbinsY=cHists[cHistIndex].GetXaxis().GetNbins()
        cYmin=cHists[cHistIndex].GetXaxis().GetXmin()
        cYmax=cHists[cHistIndex].GetXaxis().GetXmax()
        if(not cResolution2D) :
            cHistTitle='%s Resolution vs number of bits [%s scan]; Number of %s bits; %s_{KF} - %s_{TP}'%(pVarName, pDigiVarName, pDigiVarName, pVarName,pVarName)
            cResolution2D = ROOT.TH2D(cHistName,cHistTitle,cNbinsX,cXmin,cXmax,cNbinsY,cYmin,cYmax)
            cResolution2D.SetStats(0)
            cResolution2D.GetYaxis().SetTitleOffset(1.2)

        for iBin in range(1,cNbinsY) :
            cBin = cResolution2D.FindBin(cDigiValue,cHists[cHistIndex].GetBinCenter(iBin))
            cResolution2D.SetBinContent(cBin,cHists[cHistIndex].GetBinContent(iBin))

        cBin = cProfResolution.FindBin(cDigiValue)
        cProfResolution.SetBinContent( cBin, cResolution)
        cProfResolution.SetBinError( cBin, cResolutionError)
        cProfEfficiency.SetBinContent( cBin, cEff[0])
        cProfEfficiency.SetBinError( cBin, cEff[1])
    cCanvas.cd()
    cCanvas.cd(1)
    cResolution2D.DrawCopy("colz")
    cCanvas.cd(2)
    cProfResolution.SetStats(0)
    cProfResolution.DrawCopy("hist")
    cCanvas.cd(3)
    cProfEfficiency.SetStats(0)
    cProfEfficiency.DrawCopy("hist")
    return cCanvas

In [36]:
c = drawScanResult_DTC("Rt",[-4,2],"D0")
c.Draw()